In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import re
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input , Dense , LSTM , Embedding
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import EarlyStopping , ModelCheckpoint , TensorBoard , ReduceLROnPlateau


In [2]:
path = "D:/dkish/Documents/Files/MACHINELEARNING/TrichyWeatherDataWindspeed/2016/csv/20162.csv"

In [3]:
df = pd.read_csv(path , sep = ',' , header = 0 , index_col = False)

D:\ProgramFiles2\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (2,6,12,13,14,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head(20)

,Date,Time,Temp Out,Hi Temp,Low Temp,Out Hum,Dew Pt.,Wind Speed,Wind Dir,Wind Run,...,In Hum,In Dew,In Heat,In EMC,In Air Density,ET,Wind Samp,Wind Tx,ISS Recept,Arc. Int.
0,01-23-2016,05:01,77.9,77.9,77.9,77,70.1,1.8,WSW,0.11,...,50,67.9,93.5,9.05,1.1316,0.0,24,1,100.0,1
1,01-23-2016,05:02,77.9,77.9,77.9,77,70.1,2.2,SW,0.13,...,50,67.9,93.5,9.05,1.1317,0.0,23,1,100.0,1
2,01-23-2016,05:03,77.9,77.9,77.9,77,70.1,2.2,SW,0.13,...,50,67.9,93.5,9.05,1.1316,0.0,24,1,100.0,1
3,01-23-2016,05:04,77.8,77.9,77.8,77,70,2.2,SW,0.13,...,50,67.9,93.5,9.05,1.1317,0.0,23,1,100.0,1
4,01-23-2016,05:05,77.8,77.8,77.8,77,70,1.8,WSW,0.11,...,51,68.4,93.8,9.17,1.1312,0.0,23,1,100.0,1
5,01-23-2016,05:06,77.8,77.8,77.8,77,70,2.7,SW,0.16,...,50,67.9,93.5,9.05,1.1317,0.0,24,1,100.0,1
6,01-23-2016,05:07,77.8,77.8,77.8,77,70,1.3,WSW,0.08,...,51,68.4,93.8,9.17,1.1312,0.0,23,1,100.0,1
7,01-23-2016,05:08,77.8,77.8,77.8,78,70.4,1.8,SW,0.11,...,51,68.4,93.8,9.17,1.1312,0.0,24,1,100.0,1
8,01-23-2016,05:09,77.7,77.8,77.7,78,70.3,2.2,WSW,0.13,...,51,68.4,93.8,9.17,1.1312,0.0,23,1,100.0,1
9,01-23-2016,05:10,77.7,77.7,77.7,78,70.3,1.8,SW,0.11,...,51,68.4,93.8,9.17,1.1312,0.0,24,1,100.0,1


In [5]:
df.shape

(473114, 37)

In [6]:
def date_string(x):
    date = x[0]
    date = re.sub('-' , '' , date)
    
    time = x[1]
    time = re.sub(':' , '' , time)
    
    return "{0}{1}".format(date , time)

In [7]:
datestr = df[['Date' , 'Time']].apply(date_string , axis = 1)
df['DateTime'] = pd.to_datetime(datestr , format = '%m%d%Y%H%M')


In [8]:
df.head(20)

,Date,Time,Temp Out,Hi Temp,Low Temp,Out Hum,Dew Pt.,Wind Speed,Wind Dir,Wind Run,...,In Dew,In Heat,In EMC,In Air Density,ET,Wind Samp,Wind Tx,ISS Recept,Arc. Int.,DateTime
0,01-23-2016,05:01,77.9,77.9,77.9,77,70.1,1.8,WSW,0.11,...,67.9,93.5,9.05,1.1316,0.0,24,1,100.0,1,2016-01-23 05:01:00
1,01-23-2016,05:02,77.9,77.9,77.9,77,70.1,2.2,SW,0.13,...,67.9,93.5,9.05,1.1317,0.0,23,1,100.0,1,2016-01-23 05:02:00
2,01-23-2016,05:03,77.9,77.9,77.9,77,70.1,2.2,SW,0.13,...,67.9,93.5,9.05,1.1316,0.0,24,1,100.0,1,2016-01-23 05:03:00
3,01-23-2016,05:04,77.8,77.9,77.8,77,70,2.2,SW,0.13,...,67.9,93.5,9.05,1.1317,0.0,23,1,100.0,1,2016-01-23 05:04:00
4,01-23-2016,05:05,77.8,77.8,77.8,77,70,1.8,WSW,0.11,...,68.4,93.8,9.17,1.1312,0.0,23,1,100.0,1,2016-01-23 05:05:00
5,01-23-2016,05:06,77.8,77.8,77.8,77,70,2.7,SW,0.16,...,67.9,93.5,9.05,1.1317,0.0,24,1,100.0,1,2016-01-23 05:06:00
6,01-23-2016,05:07,77.8,77.8,77.8,77,70,1.3,WSW,0.08,...,68.4,93.8,9.17,1.1312,0.0,23,1,100.0,1,2016-01-23 05:07:00
7,01-23-2016,05:08,77.8,77.8,77.8,78,70.4,1.8,SW,0.11,...,68.4,93.8,9.17,1.1312,0.0,24,1,100.0,1,2016-01-23 05:08:00
8,01-23-2016,05:09,77.7,77.8,77.7,78,70.3,2.2,WSW,0.13,...,68.4,93.8,9.17,1.1312,0.0,23,1,100.0,1,2016-01-23 05:09:00
9,01-23-2016,05:10,77.7,77.7,77.7,78,70.3,1.8,SW,0.11,...,68.4,93.8,9.17,1.1312,0.0,24,1,100.0,1,2016-01-23 05:10:00


In [9]:
df.drop('Date' , axis = 1 , inplace = True)
df.drop('Time' , axis = 1 , inplace = True)
df.drop('Hi Dir' , axis = 1 , inplace = True)
df.drop('Wind Dir' , axis = 1 , inplace = True)

In [10]:
df.head(20)

,Temp Out,Hi Temp,Low Temp,Out Hum,Dew Pt.,Wind Speed,Wind Run,Hi Speed,Wind Chill,Heat Index,...,In Dew,In Heat,In EMC,In Air Density,ET,Wind Samp,Wind Tx,ISS Recept,Arc. Int.,DateTime
0,77.9,77.9,77.9,77,70.1,1.8,0.11,2.7,77.9,80.8,...,67.9,93.5,9.05,1.1316,0.0,24,1,100.0,1,2016-01-23 05:01:00
1,77.9,77.9,77.9,77,70.1,2.2,0.13,2.7,77.9,80.8,...,67.9,93.5,9.05,1.1317,0.0,23,1,100.0,1,2016-01-23 05:02:00
2,77.9,77.9,77.9,77,70.1,2.2,0.13,2.7,77.9,80.8,...,67.9,93.5,9.05,1.1316,0.0,24,1,100.0,1,2016-01-23 05:03:00
3,77.8,77.9,77.8,77,70,2.2,0.13,3.1,77.8,80.7,...,67.9,93.5,9.05,1.1317,0.0,23,1,100.0,1,2016-01-23 05:04:00
4,77.8,77.8,77.8,77,70,1.8,0.11,2.2,77.8,80.7,...,68.4,93.8,9.17,1.1312,0.0,23,1,100.0,1,2016-01-23 05:05:00
5,77.8,77.8,77.8,77,70,2.7,0.16,3.6,77.8,80.7,...,67.9,93.5,9.05,1.1317,0.0,24,1,100.0,1,2016-01-23 05:06:00
6,77.8,77.8,77.8,77,70,1.3,0.08,2.2,77.8,80.7,...,68.4,93.8,9.17,1.1312,0.0,23,1,100.0,1,2016-01-23 05:07:00
7,77.8,77.8,77.8,78,70.4,1.8,0.11,2.2,77.8,80.8,...,68.4,93.8,9.17,1.1312,0.0,24,1,100.0,1,2016-01-23 05:08:00
8,77.7,77.8,77.7,78,70.3,2.2,0.13,2.7,77.7,80.7,...,68.4,93.8,9.17,1.1312,0.0,23,1,100.0,1,2016-01-23 05:09:00
9,77.7,77.7,77.7,78,70.3,1.8,0.11,2.2,77.7,80.7,...,68.4,93.8,9.17,1.1312,0.0,24,1,100.0,1,2016-01-23 05:10:00


In [11]:
df.shape

(473114, 34)

In [12]:
df = df.set_index('DateTime')

In [13]:
df.head(20)

,Temp Out,Hi Temp,Low Temp,Out Hum,Dew Pt.,Wind Speed,Wind Run,Hi Speed,Wind Chill,Heat Index,...,In Hum,In Dew,In Heat,In EMC,In Air Density,ET,Wind Samp,Wind Tx,ISS Recept,Arc. Int.
DateTime,,,,,,,,,,,,,,,,,,,,,
2016-01-23 05:01:00,77.9,77.9,77.9,77,70.1,1.8,0.11,2.7,77.9,80.8,...,50,67.9,93.5,9.05,1.1316,0.0,24,1,100.0,1
2016-01-23 05:02:00,77.9,77.9,77.9,77,70.1,2.2,0.13,2.7,77.9,80.8,...,50,67.9,93.5,9.05,1.1317,0.0,23,1,100.0,1
2016-01-23 05:03:00,77.9,77.9,77.9,77,70.1,2.2,0.13,2.7,77.9,80.8,...,50,67.9,93.5,9.05,1.1316,0.0,24,1,100.0,1
2016-01-23 05:04:00,77.8,77.9,77.8,77,70,2.2,0.13,3.1,77.8,80.7,...,50,67.9,93.5,9.05,1.1317,0.0,23,1,100.0,1
2016-01-23 05:05:00,77.8,77.8,77.8,77,70,1.8,0.11,2.2,77.8,80.7,...,51,68.4,93.8,9.17,1.1312,0.0,23,1,100.0,1
2016-01-23 05:06:00,77.8,77.8,77.8,77,70,2.7,0.16,3.6,77.8,80.7,...,50,67.9,93.5,9.05,1.1317,0.0,24,1,100.0,1
2016-01-23 05:07:00,77.8,77.8,77.8,77,70,1.3,0.08,2.2,77.8,80.7,...,51,68.4,93.8,9.17,1.1312,0.0,23,1,100.0,1
2016-01-23 05:08:00,77.8,77.8,77.8,78,70.4,1.8,0.11,2.2,77.8,80.8,...,51,68.4,93.8,9.17,1.1312,0.0,24,1,100.0,1
2016-01-23 05:09:00,77.7,77.8,77.7,78,70.3,2.2,0.13,2.7,77.7,80.7,...,51,68.4,93.8,9.17,1.1312,0.0,23,1,100.0,1


In [14]:
df.drop('Heat D-D ' , axis = 1 , inplace = True)
df.drop('ISS Recept' , axis = 1 , inplace = True)
df.drop('Arc. Int.' , axis = 1 , inplace = True)


In [15]:
df.drop('Wind Tx ' , axis = 1 , inplace = True)


In [16]:
df.drop('ET ' , axis = 1 , inplace = True)
df.drop('Rain' , axis = 1 , inplace = True)
df.drop('Rain Rate' , axis = 1 , inplace = True)

In [17]:
df.head(20)

,Temp Out,Hi Temp,Low Temp,Out Hum,Dew Pt.,Wind Speed,Wind Run,Hi Speed,Wind Chill,Heat Index,...,UV Dose,Hi UV,Cool D-D,In Temp,In Hum,In Dew,In Heat,In EMC,In Air Density,Wind Samp
DateTime,,,,,,,,,,,,,,,,,,,,,
2016-01-23 05:01:00,77.9,77.9,77.9,77,70.1,1.8,0.11,2.7,77.9,80.8,...,0.0,0.0,0.009,88.9,50,67.9,93.5,9.05,1.1316,24
2016-01-23 05:02:00,77.9,77.9,77.9,77,70.1,2.2,0.13,2.7,77.9,80.8,...,0.0,0.0,0.009,88.9,50,67.9,93.5,9.05,1.1317,23
2016-01-23 05:03:00,77.9,77.9,77.9,77,70.1,2.2,0.13,2.7,77.9,80.8,...,0.0,0.0,0.009,88.9,50,67.9,93.5,9.05,1.1316,24
2016-01-23 05:04:00,77.8,77.9,77.8,77,70,2.2,0.13,3.1,77.8,80.7,...,0.0,0.0,0.009,88.9,50,67.9,93.5,9.05,1.1317,23
2016-01-23 05:05:00,77.8,77.8,77.8,77,70,1.8,0.11,2.2,77.8,80.7,...,0.0,0.0,0.009,88.9,51,68.4,93.8,9.17,1.1312,23
2016-01-23 05:06:00,77.8,77.8,77.8,77,70,2.7,0.16,3.6,77.8,80.7,...,0.0,0.0,0.009,88.9,50,67.9,93.5,9.05,1.1317,24
2016-01-23 05:07:00,77.8,77.8,77.8,77,70,1.3,0.08,2.2,77.8,80.7,...,0.0,0.0,0.009,88.9,51,68.4,93.8,9.17,1.1312,23
2016-01-23 05:08:00,77.8,77.8,77.8,78,70.4,1.8,0.11,2.2,77.8,80.8,...,0.0,0.0,0.009,88.9,51,68.4,93.8,9.17,1.1312,24
2016-01-23 05:09:00,77.7,77.8,77.7,78,70.3,2.2,0.13,2.7,77.7,80.7,...,0.0,0.0,0.009,88.9,51,68.4,93.8,9.17,1.1312,23


In [18]:
df.shape

(473114, 26)

In [19]:
target_names = ['Temp Out' , 'Wind Speed']

In [20]:
shift_days = 1
shift_steps = shift_days*24*60

In [21]:
df_targets = df[target_names].shift(-shift_steps)


In [22]:
df[target_names].head(shift_steps +5)

,Temp Out,Wind Speed
DateTime,,
2016-01-23 05:01:00,77.9,1.8
2016-01-23 05:02:00,77.9,2.2
2016-01-23 05:03:00,77.9,2.2
2016-01-23 05:04:00,77.8,2.2
2016-01-23 05:05:00,77.8,1.8
2016-01-23 05:06:00,77.8,2.7
2016-01-23 05:07:00,77.8,1.3
2016-01-23 05:08:00,77.8,1.8
2016-01-23 05:09:00,77.7,2.2


In [23]:
df_targets.head(5)

,Temp Out,Wind Speed
DateTime,,
2016-01-23 05:01:00,78.9,3.6
2016-01-23 05:02:00,78.8,2.2
2016-01-23 05:03:00,78.8,2.2
2016-01-23 05:04:00,78.8,2.7
2016-01-23 05:05:00,78.8,1.8


In [24]:
x_data = df.values[0:-shift_steps]

In [29]:
x_data

array([[77.9, 77.9, 77.9, ..., 9.05, 1.1316, 24],
       [77.9, 77.9, 77.9, ..., 9.05, 1.1317, 23],
       [77.9, 77.9, 77.9, ..., 9.05, 1.1316, 24],
       ...,
       [82.2, 82.2, 82.2, ..., 7.51, 1.1249, 23],
       [82.2, 82.2, 82.2, ..., 7.51, 1.1247, 24],
       [82.2, 82.3, 82.2, ..., 7.51, 1.1247, 23]], dtype=object)

In [50]:
x_data[0][0]

77.9

In [31]:
print(type(x_data))
print("Shape: " , x_data.shape)

<class 'numpy.ndarray'>
Shape:  (471674, 26)


In [32]:
y_data = df_targets.values[:-shift_steps]

In [33]:
print(type(y_data))
print("Shape:", y_data.shape)

<class 'numpy.ndarray'>
Shape: (471674, 2)


In [34]:
num_data = len(x_data)
num_data

471674

In [35]:
train_split = 0.9

In [36]:
num_train = int(train_split * num_data)
num_train

424506

In [37]:
num_test = num_data - num_train
num_test

47168

In [38]:
x_train = x_data[0:num_train]
x_test = x_data[num_train:]
len(x_train) + len(x_test)

471674

In [39]:
y_train = y_data[0:num_train]
y_test = y_data[num_train:]
len(y_train) + len(y_test)

471674

In [40]:
num_x_signals = x_data.shape[1]
num_x_signals

26

In [41]:
num_y_signals = y_data.shape[1]
num_y_signals

2

In [42]:
print("Min:", np.min(x_train))
print("Max:", np.max(x_train))

TypeError: '<=' not supported between instances of 'float' and 'str'

In [ ]:
x_scaler = MinMaxScaler()

In [ ]:
x_train_scaled = x_scaler.fit_transform(x_train)

In [ ]:
print("Min:", np.min(x_train_scaled))
print("Max:", np.max(x_train_scaled))

In [ ]:
x_test_scaled = x_scaler.transform(x_test)

In [ ]:
y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [ ]:
print(x_train_scaled.shape)
print(y_train_scaled.shape)

In [ ]:
def batch_generator(batch_size, sequence_length):
    """
    Generator function for creating random batches of training-data.
    """

    # Infinite loop.
    while True:
        # Allocate a new array for the batch of input-signals.
        x_shape = (batch_size, sequence_length, num_x_signals)
        x_batch = np.zeros(shape=x_shape, dtype=np.float16)

        # Allocate a new array for the batch of output-signals.
        y_shape = (batch_size, sequence_length, num_y_signals)
        y_batch = np.zeros(shape=y_shape, dtype=np.float16)

        # Fill the batch with random sequences of data.
        for i in range(batch_size):
            # Get a random start-index.
            # This points somewhere into the training-data.
            idx = np.random.randint(num_train - sequence_length)
            
            # Copy the sequences of data starting at this index.
            x_batch[i] = x_train_scaled[idx:idx+sequence_length]
            y_batch[i] = y_train_scaled[idx:idx+sequence_length]
        
        yield (x_batch, y_batch)

In [ ]:
batch_size = 31

In [ ]:
sequence_length = 24 * 60
sequence_length

In [ ]:
generator = batch_generator(batch_size=batch_size,
                            sequence_length=sequence_length)

In [ ]:
x_batch, y_batch = next(generator)

In [ ]:
print(x_batch.shape)
print(y_batch.shape)

In [ ]:
validation_data = (np.expand_dims(x_test_scaled, axis=0),
                   np.expand_dims(y_test_scaled, axis=0))

In [ ]:
model = Sequential()

In [ ]:
model.add(GRU(units=512,
              return_sequences=True,
              input_shape=(None, num_x_signals,)))

In [ ]:
model.add(Dense(num_y_signals, activation='sigmoid'))

In [ ]:
warmup_steps = 50

In [ ]:
def loss_mse_warmup(y_true, y_pred):
    """
    Calculate the Mean Squared Error between y_true and y_pred,
    but ignore the beginning "warmup" part of the sequences.
    
    y_true is the desired output.
    y_pred is the model's output.
    """

    # The shape of both input tensors are:
    # [batch_size, sequence_length, num_y_signals].

    # Ignore the "warmup" parts of the sequences
    # by taking slices of the tensors.
    y_true_slice = y_true[:, warmup_steps:, :]
    y_pred_slice = y_pred[:, warmup_steps:, :]

    # These sliced tensors both have this shape:
    # [batch_size, sequence_length - warmup_steps, num_y_signals]

    # Calculate the MSE loss for each value in these tensors.
    # This outputs a 3-rank tensor of the same shape.
    loss = tf.losses.mean_squared_error(labels=y_true_slice,
                                        predictions=y_pred_slice)

    # Keras may reduce this across the first axis (the batch)
    # but the semantics are unclear, so to be sure we use
    # the loss across the entire tensor, we reduce it to a
    # single scalar with the mean function.
    loss_mean = tf.reduce_mean(loss)

    return loss_mean

In [ ]:
optimizer = RMSprop(lr=1e-3)

In [ ]:
model.compile(loss=loss_mse_warmup, optimizer=optimizer,metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
path_checkpoint = '23_checkpoint.keras'
callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                      monitor='val_loss',
                                      verbose=1,
                                      save_weights_only=True,
                                      save_best_only=True)

In [ ]:
callback_early_stopping = EarlyStopping(monitor='val_loss',
                                        patience=5, verbose=1)

In [ ]:
callback_tensorboard = TensorBoard(log_dir='./23_logs/',
                                   histogram_freq=0,
                                   write_graph=False)

In [ ]:
callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                       factor=0.1,
                                       min_lr=1e-4,
                                       patience=0,
                                       verbose=1)

In [ ]:
%%time
model.fit_generator(generator=generator,
                    epochs=50,
                    steps_per_epoch=100,
                    validation_data=validation_data,
                    callbacks=callbacks)